# Training A Neural Netowrk Using PyTorch and My Custom Libraries
This notebook will go through and explain the steps involved in creating a NN model and training it given some dataset

## Composer Pattern
Model training is set-up using the composer pattern. Where, you create different components and compose them onto a ModelTrainer, which ultimately takes care of the training/validation process. 
![Pics](./composer.png)


## Setting GPU
The first step for systems with multiple GPUs is to set which one we want to use. In torch, this is done by changing the environment varaible **CUDA_VISIBLE_DEVICES**. This makes cuda only 'see' that one specific GPU. i.e., all data will be loaded to that GPU. Pytorch will then regard this GPU as 'cuda:0'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## Defining a Dataset
Let's define a random dataset. Most of our data is stored as pandas dataframes. I have convinience functions that can convert these DataFrames onto Pytorch DataLoaders. Which, as the name suggests, fetches data during model training. However, you can also manually define your own DataLoader class to use some other types of data.

In [3]:
import pandas as pd
from inverse_modelling_tfo.data.data_loader_gen import generate_data_loaders

dummy_data = pd.DataFrame({
        'A': [1, 2, 3, 4, 5],
        'B': [2, 3, 4, 5, 6],
        'C': [3, 5, 7, 9, 11]})
data_loader_params = {
    'batch_size': 2,    # The size of the batches that the dataloader will output
    'shuffle': True,    # The dataloader will shuffle its outputs at each epoch
    'num_workers': 0,   # The number of workers that the dataloader will use to generate the batches
}
train_loader, validation_loader = generate_data_loaders(dummy_data, data_loader_params, ['A', 'B'], ['C'])
print("The output type is", type(train_loader))

The output type is <class 'torch.utils.data.dataloader.DataLoader'>


__Notes__: The data generated from these loaders are by default set to be stored on CUDA. More specifically, the first GPU on CUDA. You can change the device parameter on **generate_data_loaders** to set this

## Validation Method
You can pass different validation strategies to alter the training and validation loader behavior. By default, the validation strategy is a RandomSplit(0.8). But, we have a few more options. Here are a few examples. For more info, check the test folder

In [4]:
from inverse_modelling_tfo.models.validation_methods import RandomSplit, ValidationMethod, HoldOneOut, CVSplit, CombineMethods

train_loader, validation_loader = generate_data_loaders(dummy_data, data_loader_params, ['A', 'B'], ['C'], CVSplit(2, 0))

# Loss Function
The next component we need for training a model is a LossFunction. I have a custom wrapper around pytorch's native loss functions. The benefits of this are two-fold. This wrapper allows us to track the losses over each step and each epoch of the training. Making for a far easier loss tracking/plotting expreience. Additionally, this extends the loss function to include more exotic losses which can take extra inputs(other than model labels and preidctions). This includes physics losses.

In [5]:
from inverse_modelling_tfo.models import TorchLossWrapper
import torch.nn as nn   # PyTorch's neural network module

criterion = TorchLossWrapper(nn.MSELoss(), name="label_loss")

# Model Trainer and Model Trainer Factory
Training is done using a ModelTrainer module. This makes training/validation both easier and allows us to keep track of the setup as well for reporducibility. ModelTrainer are usually created from the Factory class, which allows us to put all the parameters in before hand. One last piece we need is the model architecture. We have a custom_models file containing a bunch of custom Pytorch models which are much faster/easier to initialize.    

The model example we use here is a fully connected perceptron with BatchNormalization and Dropout layers, called PerceptronBD. The 'node_counts' here 

In [ ]:
from inverse_modelling_tfo.models import ModelTrainerFactory
from inverse_modelling_tfo.models.custom_models import PerceptronBD

trainer_factory = ModelTrainerFactory(
    PerceptronBD, {"node_counts": [2, 2, 1]}, generate_data_loaders, data_loader_params, 5, criterion
)